In [46]:
import pandas as pd
from scipy import sparse

In [47]:
ratings = pd.read_csv("dummy.txt",index_col=0)
ratings

,horror1,horror2,horror3,comedy1,comedy2,comedy3
user 1,4.0,5.0,3.0,NaN,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,NaN
user 3,1.0,NaN,NaN,4.0,5.0,4.0
user 4,NaN,2.0,1.0,4.0,NaN,3.0
user 5,1.0,NaN,2.0,3.0,3.0,4.0


In [48]:
ratings = ratings.fillna(0)
ratings

,horror1,horror2,horror3,comedy1,comedy2,comedy3
user 1,4.0,5.0,3.0,0.0,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,0.0
user 3,1.0,0.0,0.0,4.0,5.0,4.0
user 4,0.0,2.0,1.0,4.0,0.0,3.0
user 5,1.0,0.0,2.0,3.0,3.0,4.0


In [49]:
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row
ratings_std = ratings.apply(standardize).T
print(ratings_std)


sparse_df = sparse.csr_matrix(ratings_std.values)
corrMatrix = pd.DataFrame(cosine_similarity(sparse_df),index=ratings.columns,columns=ratings.columns)
corrMatrix


         user 1  user 2  user 3    user 4    user 5
horror1    0.36    0.56   -0.24 -0.440000 -0.240000
horror2    0.60    0.20   -0.40  0.000000 -0.400000
horror3    0.40    0.40   -0.60 -0.266667  0.066667
comedy1   -0.65   -0.15    0.35  0.350000  0.100000
comedy2   -0.08   -0.08    0.52 -0.480000  0.120000
comedy3   -0.35   -0.60    0.40  0.150000  0.400000


,horror1,horror2,horror3,comedy1,comedy2,comedy3
horror1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
horror2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
horror3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
comedy1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
comedy2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939
comedy3,-0.914106,-0.843374,-0.802181,0.723747,0.393939,1.000000


In [50]:
corrMatrix = ratings.corr(method='pearson')
corrMatrix.head(6)

,horror1,horror2,horror3,comedy1,comedy2,comedy3
horror1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
horror2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
horror3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
comedy1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
comedy2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939
comedy3,-0.914106,-0.843374,-0.802181,0.723747,0.393939,1.000000


In [51]:
def get_similar(movie_name,rating):
    similar_score = corrMatrix[movie_name]*(rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

In [52]:
item_similarity

array([[ 1.        ,  0.70668875,  0.81368151, -0.79941088, -0.02539184,
        -0.91410609],
       [ 0.70668875,  1.        ,  0.72310153, -0.84515425, -0.5189993 ,
        -0.84337386],
       [ 0.81368151,  0.72310153,  1.        , -0.84794611, -0.3799803 ,
        -0.80218063],
       [-0.79941088, -0.84515425, -0.84794611,  1.        ,  0.14803913,
         0.72374686],
       [-0.02539184, -0.5189993 , -0.3799803 ,  0.14803913,  1.        ,
         0.39393939],
       [-0.91410609, -0.84337386, -0.80218063,  0.72374686,  0.39393939,
         1.        ]])

In [53]:
print(item_similarity)

[[ 1.          0.70668875  0.81368151 -0.79941088 -0.02539184 -0.91410609]
 [ 0.70668875  1.          0.72310153 -0.84515425 -0.5189993  -0.84337386]
 [ 0.81368151  0.72310153  1.         -0.84794611 -0.3799803  -0.80218063]
 [-0.79941088 -0.84515425 -0.84794611  1.          0.14803913  0.72374686]
 [-0.02539184 -0.5189993  -0.3799803   0.14803913  1.          0.39393939]
 [-0.91410609 -0.84337386 -0.80218063  0.72374686  0.39393939  1.        ]]


In [54]:
#converting numpy array into data frame
item_similarity_df = pd.DataFrame(item_similarity,index=ratings.columns,columns=ratings.columns)
item_similarity_df

,horror1,horror2,horror3,comedy1,comedy2,comedy3
horror1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
horror2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
horror3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
comedy1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
comedy2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939
comedy3,-0.914106,-0.843374,-0.802181,0.723747,0.393939,1.000000


In [55]:
#let's make recommendations
def get_similar_movies(movie_name,user_rating):
    similar_score = item_similarity_df[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score
print (get_similar_movies("comedy3",1))

horror1    1.371159
horror2    1.265061
horror3    1.203271
comedy2   -0.590909
comedy1   -1.085620
comedy3   -1.500000
Name: comedy3, dtype: float64


In [56]:
user1 = [("horror1",5),("horror2",1),("horror3",1)]
similar_movies = pd.DataFrame()
for movie,rating in user1:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating),ignore_index=True)

In [57]:
similar_movies.head()

,comedy1,comedy2,comedy3,horror1,horror2,horror3
0,-1.998527,-0.063480,-2.285265,2.500000,1.766722,2.034204
1,1.267731,0.778499,1.265061,-1.060033,-1.500000,-1.084652
2,1.271919,0.569970,1.203271,-1.220522,-1.084652,-1.500000


In [58]:
similar_movies.sum().sort_values(ascending=False)

comedy2    1.284990
comedy1    0.541123
horror1    0.219445
comedy3    0.183067
horror3   -0.550449
horror2   -0.817930
dtype: float64